In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

### 0. Setting for training

In [2]:
batch_size = 64

### 1. Prepare Data

In [3]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='../PyTorchZeroToAll/data/', train=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='../PyTorchZeroToAll/data/', train=False, transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### 2. Define Model

In [6]:
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)
        
    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)
    

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

model = Net()

### 3. Construct Loss function and Optimizer

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

### 4. Train & Test

In [9]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298378
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.295555
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.291525
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.293450
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.305683
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.302148
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.308975
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.303850
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.293193
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.296624
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.286069
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.294731
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.288919
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.287816
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.290276
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.278593
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.270977
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.250397
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.239914
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.214410
Train Epoch: 1 [12800/60000 (

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.2130, Accuracy: 9360/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.224395
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.131339
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.132259
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.330344
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.197786
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.282052
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.252202
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.215216
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.073863
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.347182
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.186170
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.124612
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.184957
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.054414
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.209661
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.239733
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.105545
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.359790
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.147759
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.048892
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.102547
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.024674
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.082690
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.047660
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.218130
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.064826
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.138562
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.099086
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.068062
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.042102
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.041296
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.030058
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.016817
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.157869
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.055456
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.080391
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.097871
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.116869
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.084775


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.201670
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.133017
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.157975
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.038343
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.122619
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.089918
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.020051
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.049769
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.013902
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.073988
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.203455
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.084099
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.020764
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.102095
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.141071
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.014348
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.133845
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.025759
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.009896
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.049407


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.031365
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.070380
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.009073
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.096018
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.123387
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.073446
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.050394
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.096394
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.091650
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.051234
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.109429
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.022299
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.026793
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.065218
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.019624
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.064305
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.016674
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.080161
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.088035
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.138373


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.025424
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.098357
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.055666
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.122628
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.081588

Test set: Average loss: 0.0503, Accuracy: 9849/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.104190
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.023908
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.026441
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.015577
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.028426
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.022666
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.016819
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.078277
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.055717
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.123256
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.030650
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.157215
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.004865
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.092702
Train Epoch: 9 [